In [40]:
import numpy as np
import pandas as pd
table = 'prevApp'


In [2]:
app_train = pd.read_csv('../1_data_import/application_train.csv')
prev_app = pd.read_csv('../1_data_import/previous_application.csv')

In [15]:
# get rid of system errors

prev_app = prev_app[prev_app.FLAG_LAST_APPL_PER_CONTRACT != 'N']
prev_app = prev_app[prev_app.NFLAG_LAST_APPL_IN_DAY != 0]

# get rid of columns helping identified system errors
columns = ['FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY']
prev_app = prev_app.drop(columns, axis=1)

In [29]:
#one-hot encoding
#NAME_CONTRACT_TYPE
#WEEKDAY_APPR_PROCESS_START
#HOUR_APPR_PROCESS_START b.duzo unikalnych wartosci
#NAME_SELLER_INDUSTRY b.duzo unikalnych wartosci

In [83]:
rangeFunc = lambda x: x.max() - x.min()
PREVIOUS_APPLICATION_AGGREGATION_RECIPIES = []
for agg in ['mean', 'min', 'max', 'sum', 'var', 'std']:
    for select in ['AMT_ANNUITY',
                   'AMT_APPLICATION',
                   'AMT_CREDIT',
                   'AMT_DOWN_PAYMENT',
                   'AMT_GOODS_PRICE',
                   'CNT_PAYMENT',
                   'DAYS_DECISION',
                   'HOUR_APPR_PROCESS_START',
                   'RATE_DOWN_PAYMENT'
                   ]:
        PREVIOUS_APPLICATION_AGGREGATION_RECIPIES.append((select, agg))
PREVIOUS_APPLICATION_AGGREGATION_RECIPIES = [(['SK_ID_CURR'], PREVIOUS_APPLICATION_AGGREGATION_RECIPIES)]

In [85]:
application = app_train[['SK_ID_CURR']]
groupby_aggregate_names = []
groupby_cols = PREVIOUS_APPLICATION_AGGREGATION_RECIPIES[0][0]
specs = PREVIOUS_APPLICATION_AGGREGATION_RECIPIES[0][1]
group_object = prev_app.groupby(groupby_cols)
for select, agg in specs:
    groupby_aggregate_name = table + '_' + agg + '_' + select
    application = application.merge(group_object[select]
                          .agg(agg)
                          .reset_index()
                          .rename(index=str,
                                  columns={select: groupby_aggregate_name})
                          [groupby_cols + [groupby_aggregate_name]],
                          on = groupby_cols,
                          how = 'left')
    groupby_aggregate_names.append(groupby_aggregate_name)

application.head()

,SK_ID_CURR,prevApp_mean_AMT_ANNUITY,prevApp_mean_AMT_APPLICATION,prevApp_mean_AMT_CREDIT,prevApp_mean_AMT_DOWN_PAYMENT,prevApp_mean_AMT_GOODS_PRICE,prevApp_mean_CNT_PAYMENT,prevApp_mean_DAYS_DECISION,prevApp_mean_HOUR_APPR_PROCESS_START,prevApp_mean_RATE_DOWN_PAYMENT,...,prevApp_var_RATE_DOWN_PAYMENT,prevApp_std_AMT_ANNUITY,prevApp_std_AMT_APPLICATION,prevApp_std_AMT_CREDIT,prevApp_std_AMT_DOWN_PAYMENT,prevApp_std_AMT_GOODS_PRICE,prevApp_std_CNT_PAYMENT,prevApp_std_DAYS_DECISION,prevApp_std_HOUR_APPR_PROCESS_START,prevApp_std_RATE_DOWN_PAYMENT
0,100002,9251.775,179055.00,179055.00,0.00,179055.00,24.000000,-606.000000,9.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,56553.990,435436.50,484191.00,3442.50,435436.50,10.000000,-1305.000000,14.666667,0.050030,...,0.005006,46332.557777,424161.620549,497949.861808,4868.430188,424161.620549,3.464102,898.138631,2.516611,0.070754
2,100004,5357.250,24282.00,20106.00,4860.00,24282.00,4.000000,-815.000000,5.000000,0.212008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,23651.175,272203.26,291695.50,34840.17,408304.89,23.000000,-272.444444,14.666667,0.163412,...,0.005923,13623.580119,286175.129541,333337.354853,45462.482973,253670.602059,20.228692,157.236375,1.000000,0.076958
4,100007,12278.805,150530.25,166638.75,3390.75,150530.25,20.666667,-1222.833333,12.333333,0.159516,...,0.007050,8063.586466,100585.988276,118032.409509,404.111525,100585.988276,14.348054,717.268546,2.503331,0.083967


In [89]:
# applying range function
PREVIOUS_APPLICATION_AGGREGATION_RANGE = []
for col in ['AMT_ANNUITY',
                   'AMT_APPLICATION',
                   'AMT_CREDIT',
                   'AMT_DOWN_PAYMENT',
                   'AMT_GOODS_PRICE',
                   'CNT_PAYMENT',
                   'DAYS_DECISION',
                   'HOUR_APPR_PROCESS_START',
                   'RATE_DOWN_PAYMENT'
                   ]:
        PREVIOUS_APPLICATION_AGGREGATION_RANGE.append((col, 'rangeFunc'))

groupby_aggregate_names = []

group_object = prev_app.groupby(groupby_cols)
for select, agg in PREVIOUS_APPLICATION_AGGREGATION_RANGE:
    groupby_aggregate_name = table + '_' + agg + '_' + select
    application = application.merge(group_object[select]
                          .agg(rangeFunc)
                          .reset_index()
                          .rename(index=str,
                                  columns={select: groupby_aggregate_name})
                          [groupby_cols + [groupby_aggregate_name]],
                          on = groupby_cols,
                          how = 'left')
    groupby_aggregate_names.append(groupby_aggregate_name)

application.head()

,SK_ID_CURR,prevApp_mean_AMT_ANNUITY,prevApp_mean_AMT_APPLICATION,prevApp_mean_AMT_CREDIT,prevApp_mean_AMT_DOWN_PAYMENT,prevApp_mean_AMT_GOODS_PRICE,prevApp_mean_CNT_PAYMENT,prevApp_mean_DAYS_DECISION,prevApp_mean_HOUR_APPR_PROCESS_START,prevApp_mean_RATE_DOWN_PAYMENT,...,prevApp_std_RATE_DOWN_PAYMENT,prevApp_rangeFunc_AMT_ANNUITY,prevApp_rangeFunc_AMT_APPLICATION,prevApp_rangeFunc_AMT_CREDIT,prevApp_rangeFunc_AMT_DOWN_PAYMENT,prevApp_rangeFunc_AMT_GOODS_PRICE,prevApp_rangeFunc_CNT_PAYMENT,prevApp_rangeFunc_DAYS_DECISION,prevApp_rangeFunc_HOUR_APPR_PROCESS_START,prevApp_rangeFunc_RATE_DOWN_PAYMENT
0,100002,9251.775,179055.00,179055.00,0.00,179055.00,24.000000,-606.000000,9.000000,0.000000,...,NaN,0.000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.000000
1,100003,56553.990,435436.50,484191.00,3442.50,435436.50,10.000000,-1305.000000,14.666667,0.050030,...,0.070754,91619.685,831190.5,967828.5,6885.00,831190.50,6.0,1595.0,5.0,0.100061
2,100004,5357.250,24282.00,20106.00,4860.00,24282.00,4.000000,-815.000000,5.000000,0.212008,...,NaN,0.000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.000000
3,100006,23651.175,272203.26,291695.50,34840.17,408304.89,23.000000,-272.444444,14.666667,0.163412,...,0.076958,37471.590,688500.0,906615.0,64293.66,661587.66,48.0,436.0,3.0,0.108836
4,100007,12278.805,150530.25,166638.75,3390.75,150530.25,20.666667,-1222.833333,12.333333,0.159516,...,0.083967,20844.495,230323.5,269784.0,571.50,230323.50,38.0,1983.0,7.0,0.118747
